In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/datast/cleaned_dataset.csv")

In [ ]:
df = df.astype('int64')

In [ ]:
from sklearn.preprocessing import StandardScaler

numerical_cols = ['Amount', 'Price', 'Area']

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [ ]:
x = np.array(df.drop(['Price'], axis=1))
y = np.array(df['Price'])

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

class EntropicOptimalTransportNPMLE:
    """
    Non-Parametric Maximum Likelihood Estimation using Entropic Optimal Transport
    Based on Weed & Rigollet (2018) "A notion of duality between entropy and the central limit theorem"
    """
    
    def __init__(self, reg_param=0.05, max_iter=200, tol=1e-5, 
                 max_support_size=25, bandwidth='scott'):
        """
        Parameters:
        -----------
        reg_param : float
            Entropic regularization parameter (epsilon in Sinkhorn)
        max_iter : int
            Maximum iterations for Sinkhorn algorithm
        tol : float
            Convergence tolerance
        max_support_size : int
            Maximum number of support points to consider
        bandwidth : str or float
            Bandwidth for Gaussian kernels ('scott', 'silverman', or float)
        """
        self.reg_param = reg_param
        self.max_iter = max_iter
        self.tol = tol
        self.max_support_size = max_support_size
        self.bandwidth = bandwidth
        
        # Fitted parameters
        self.support_points_ = None
        self.weights_ = None
        self.n_components_ = 0
        self.cost_matrix_ = None
        self.transport_plan_ = None
        
    def _compute_bandwidth(self, X):
        """Compute bandwidth for Gaussian kernels"""
        n_samples, n_features = X.shape
        
        if self.bandwidth == 'scott':
            # Scott's rule
            return n_samples ** (-1.0 / (n_features + 4))
        elif self.bandwidth == 'silverman':
            # Silverman's rule
            return (n_samples * (n_features + 2) / 4.0) ** (-1.0 / (n_features + 4))
        else:
            return float(self.bandwidth)
    
    def _select_support_points(self, X):
        """
        Select candidate support points using K-means clustering
        This reduces computational complexity while maintaining coverage
        """
        n_samples = X.shape[0]
        
        if n_samples <= self.max_support_size:
            return X.copy()
        
        # Use K-means to select representative points
        kmeans = KMeans(n_clusters=self.max_support_size, random_state=42, n_init=10)
        kmeans.fit(X)
        
        return kmeans.cluster_centers_
    
    def _compute_cost_matrix(self, X, Y, bandwidth):
        """
        Compute cost matrix for optimal transport
        Uses squared Euclidean distance scaled by bandwidth
        """
        # Squared Euclidean distances
        distances_sq = cdist(X, Y, metric='sqeuclidean')
        
        # Scale by bandwidth
        cost_matrix = distances_sq / (2 * bandwidth**2)
        
        return cost_matrix
    
    def _sinkhorn_algorithm(self, cost_matrix, a, b, reg_param, max_iter, tol):
        """
        Sinkhorn algorithm for entropic optimal transport
        
        Parameters:
        -----------
        cost_matrix : array, shape (n, m)
            Cost matrix C[i,j] = cost of transporting from i to j
        a : array, shape (n,)
            Source distribution (marginal constraint)
        b : array, shape (m,)
            Target distribution (marginal constraint)
        reg_param : float
            Entropic regularization parameter
        """
        n, m = cost_matrix.shape
        
        # Initialize dual variables
        u = np.ones(n) / n
        v = np.ones(m) / m
        
        # Kernel matrix K = exp(-C/reg_param)
        K = np.exp(-cost_matrix / reg_param)
        
        # Avoid numerical issues
        K = np.maximum(K, 1e-100)
        
        for iteration in range(max_iter):
            u_prev = u.copy()
            
            # Update u
            u = a / (K @ v)
            u = np.maximum(u, 1e-100)  # Avoid division by zero
            
            # Update v
            v = b / (K.T @ u)
            v = np.maximum(v, 1e-100)  # Avoid division by zero
            
            # Check convergence
            if np.linalg.norm(u - u_prev) < tol:
                break
        
        # Compute transport plan
        transport_plan = np.diag(u) @ K @ np.diag(v)
        
        return transport_plan, u, v
    
    def _gaussian_kernel_density(self, X, support_points, bandwidth):
        """
        Compute Gaussian kernel density at support points
        """
        n_samples, n_features = X.shape
        n_support = len(support_points)
        
        densities = np.zeros(n_support)
        
        for i, support_point in enumerate(support_points):
            # Compute Gaussian kernel density
            diff = X - support_point
            distances_sq = np.sum(diff**2, axis=1)
            
            # Gaussian kernel
            kernel_values = np.exp(-distances_sq / (2 * bandwidth**2))
            normalizing_constant = (2 * np.pi * bandwidth**2) ** (n_features / 2)
            
            densities[i] = np.mean(kernel_values) / normalizing_constant
        
        return densities
    
    def _extract_mixture_components(self, transport_plan, support_points, threshold=1e-6):
        """
        Extract Gaussian mixture components from transport plan
        """
        n_samples, n_support = transport_plan.shape
        
        # Sum transport plan over samples to get weights at support points
        support_weights = np.sum(transport_plan, axis=0)
        
        # Keep only support points with significant weight
        significant_indices = support_weights > threshold
        active_support_points = support_points[significant_indices]
        active_weights = support_weights[significant_indices]
        
        # Normalize weights
        if len(active_weights) > 0:
            active_weights = active_weights / np.sum(active_weights)
        
        return active_support_points, active_weights
    
    def fit(self, X, y=None):
        """
        Fit NPMLE using Entropic Optimal Transport
        """
        n_samples, n_features = X.shape
        
        # Standardize data
        self.scaler_ = StandardScaler()
        X_scaled = self.scaler_.fit_transform(X)
        
        print(f"Fitting EOT-NPMLE on {n_samples} samples with {n_features} features")
        
        # Step 1: Select support points
        print("Selecting support points...")
        candidate_support_points = self._select_support_points(X_scaled)
        n_support = len(candidate_support_points)
        print(f"Selected {n_support} candidate support points")
        
        # Step 2: Compute bandwidth
        bandwidth = self._compute_bandwidth(X_scaled)
        print(f"Computed bandwidth: {bandwidth:.6f}")
        
        # Step 3: Compute cost matrix
        print("Computing cost matrix...")
        self.cost_matrix_ = self._compute_cost_matrix(
            X_scaled, candidate_support_points, bandwidth
        )
        
        # Step 4: Set up marginal constraints
        # Source: uniform distribution over data points
        a = np.ones(n_samples) / n_samples
        
        # Target: Gaussian kernel density at support points
        b = self._gaussian_kernel_density(X_scaled, candidate_support_points, bandwidth)
        b = b / np.sum(b)  # Normalize
        
        # Step 5: Solve entropic optimal transport
        print("Solving entropic optimal transport...")
        self.transport_plan_, u, v = self._sinkhorn_algorithm(
            self.cost_matrix_, a, b, self.reg_param, self.max_iter, self.tol
        )
        
        # Step 6: Extract mixture components
        print("Extracting mixture components...")
        active_support_points, active_weights = self._extract_mixture_components(
            self.transport_plan_, candidate_support_points
        )
        
        # Step 7: Estimate covariance for each component
        components = []
        final_weights = []
        
        for i, (support_point, weight) in enumerate(zip(active_support_points, active_weights)):
            if weight > 1e-6:  # Only keep significant components
                # Estimate local covariance using weighted samples
                distances = np.sum((X_scaled - support_point)**2, axis=1)
                
                # Use transport plan to weight nearby points
                transport_weights = self.transport_plan_[:, i] if i < self.transport_plan_.shape[1] else None
                
                if transport_weights is not None and np.sum(transport_weights) > 1e-6:
                    # Weighted covariance
                    transport_weights = transport_weights / np.sum(transport_weights)
                    centered = X_scaled - support_point
                    cov = np.average(centered[:, :, np.newaxis] * centered[:, np.newaxis, :], 
                                   weights=transport_weights, axis=0)
                else:
                    # Local covariance using k-nearest neighbors
                    k_nearest = min(50, n_samples)
                    nearest_indices = np.argsort(distances)[:k_nearest]
                    nearest_points = X_scaled[nearest_indices]
                    cov = np.cov(nearest_points.T)
                
                # Add regularization
                cov += 1e-6 * np.eye(n_features)
                
                components.append({
                    'mean': support_point,
                    'cov': cov
                })
                final_weights.append(weight)
        
        # Store results
        self.support_points_ = components
        self.weights_ = final_weights
        self.n_components_ = len(components)
        
        print(f"Found {self.n_components_} components")
        print(f"Component weights: {[f'{w:.4f}' for w in self.weights_]}")
        
        return self
    
    def predict_proba(self, X):
        """Predict class probabilities"""
        if self.support_points_ is None:
            raise ValueError("Model not fitted yet")
        
        X_scaled = self.scaler_.transform(X)
        n_samples = X.shape[0]
        probabilities = np.zeros((n_samples, self.n_components_))
        
        for i in range(n_samples):
            total_prob = 0
            probs = []
            
            for k in range(self.n_components_):
                try:
                    prob = self.weights_[k] * multivariate_normal.pdf(
                        X_scaled[i],
                        self.support_points_[k]['mean'],
                        self.support_points_[k]['cov']
                    )
                    probs.append(prob)
                    total_prob += prob
                except:
                    probs.append(1e-10)
                    total_prob += 1e-10
            
            for k in range(self.n_components_):
                probabilities[i, k] = probs[k] / max(total_prob, 1e-10)
        
        return probabilities
    
    def predict(self, X):
        """Predict cluster labels"""
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)
    
    def score(self, X):
        """Compute log-likelihood"""
        X_scaled = self.scaler_.transform(X)
        n_samples = X.shape[0]
        log_likelihood = 0
        
        for i in range(n_samples):
            mixture_density = 0
            for k in range(self.n_components_):
                try:
                    density = self.weights_[k] * multivariate_normal.pdf(
                        X_scaled[i],
                        self.support_points_[k]['mean'],
                        self.support_points_[k]['cov']
                    )
                    mixture_density += density
                except:
                    mixture_density += 1e-10
            
            log_likelihood += np.log(max(mixture_density, 1e-10))
        
        return log_likelihood

def fit_eot(X, y=None, reg_param=0.05):
    """
    Function to fit EOT-NPMLE on your actual data (159630, 40)
    
    Parameters:
    -----------
    X : array-like, shape (159630, 40)
        Your feature data
    y : array-like, shape (159630,), optional
        Your target data
    reg_param : float
        Entropic regularization parameter (smaller = less regularization)
    """
    
    print(f"Fitting EOT-NPMLE on data with shape: {X.shape}")
    
    # For large datasets, use PCA for dimensionality reduction
    if X.shape[1] > 20:
        print("Applying PCA for dimensionality reduction...")
        pca = PCA(n_components=15, random_state=42)
        X_reduced = pca.fit_transform(X)
        print(f"Reduced dimensionality from {X.shape[1]} to {X_reduced.shape[1]}")
        print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.4f}")
    else:
        X_reduced = X
        pca = None
    
    # Initialize model with parameters suitable for large datasets
    eot_npmle = EntropicOptimalTransportNPMLE(
        reg_param=reg_param,  # Entropic regularization
        max_iter=500,         # Reduced for efficiency
        tol=1e-5,            # Slightly relaxed tolerance
        max_support_size=15, # Limit support points for computational efficiency
        bandwidth='scott'     # Automatic bandwidth selection
    )
    
    # Fit the model
    eot_npmle.fit(X_reduced, y)
    eot_npmle.pca_ = pca  # Store PCA for predictions
    
    # Get results
    def predict_original(X_orig):
        if pca is not None:
            X_transformed = pca.transform(X_orig)
            return eot_npmle.predict(X_transformed)
        else:
            return eot_npmle.predict(X_orig)
    
    def predict_proba_original(X_orig):
        if pca is not None:
            X_transformed = pca.transform(X_orig)
            return eot_npmle.predict_proba(X_transformed)
        else:
            return eot_npmle.predict_proba(X_orig)
    
    cluster_labels = predict_original(X)
    probabilities = predict_proba_original(X)
    
    print(f"\nResults:")
    print(f"Number of components found: {eot_npmle.n_components_}")
    print(f"Component weights: {[f'{w:.4f}' for w in eot_npmle.weights_]}")
    print(f"Unique cluster labels: {np.unique(cluster_labels)}")
    print(f"Cluster distribution: {np.bincount(cluster_labels)}")
    
    return {
        'model': eot_npmle,
        'pca': pca,
        'cluster_labels': cluster_labels,
        'probabilities': probabilities,
        'n_components': eot_npmle.n_components_,
        'weights': eot_npmle.weights_,
        'predict_fn': predict_original,
        'predict_proba_fn': predict_proba_original
    }


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

def evaluate_eot_with_gboost(X, y, n_splits=5, reg_param=0.1, random_state=42):
    """
    Evaluate EOT-NPMLE for regression using Gradient Boosting and PCA on cluster probabilities.

    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
        Feature matrix
    y : array-like, shape (n_samples,)
        Target values
    n_splits : int
        Number of cross-validation folds
    reg_param : float
        Entropic regularization parameter for EOT
    random_state : int
        Random seed for reproducibility

    Returns:
    --------
    results : dict
        Evaluation metrics and models per fold
    """

    results = {
        'mse': [], 'r2': [], 'mae': [],
        'models': [], 'test_preds': [], 'test_true': []
    }

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"\n=== Fold {fold + 1}/{n_splits} ===")
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Fit EOT-NPMLE
        print("Fitting EOT-NPMLE...")
        eot_result = fit_eot(X_train, y_train, reg_param=reg_param)

        # Get cluster probabilities
        train_probs = eot_result['probabilities']
        test_probs = eot_result['predict_proba_fn'](X_test)

        # Reduce dimensions of probability features
        print("Applying PCA...")
        pca = PCA(n_components=8, random_state=random_state)
        train_pca = pca.fit_transform(train_probs)
        test_pca = pca.transform(test_probs)

        # Train Gradient Boosting Regressor
        print("Training Gradient Boosting model...")
        gboost = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=random_state)
        gboost.fit(train_pca, y_train)

        # Predict
        y_pred = gboost.predict(test_pca)

        # Evaluate
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        print(f"Fold {fold + 1} Results:")
        print(f"- MSE: {mse:.4f}")
        print(f"- R²: {r2:.4f}")
        print(f"- MAE: {mae:.4f}")

        results['mse'].append(mse)
        results['r2'].append(r2)
        results['mae'].append(mae)
        results['models'].append((eot_result['model'], gboost))
        results['test_preds'].append(y_pred)
        results['test_true'].append(y_test)

    # Summary
    results['mean_mse'] = np.mean(results['mse'])
    results['std_mse'] = np.std(results['mse'])
    results['mean_r2'] = np.mean(results['r2'])
    results['std_r2'] = np.std(results['r2'])
    results['mean_mae'] = np.mean(results['mae'])
    results['std_mae'] = np.std(results['mae'])

    print("\n=== Final Evaluation ===")
    print(f"Average MSE: {results['mean_mse']:.4f} ± {results['std_mse']:.4f}")
    print(f"Average R²: {results['mean_r2']:.4f} ± {results['std_r2']:.4f}")
    print(f"Average MAE: {results['mean_mae']:.4f} ± {results['std_mae']:.4f}")

    return results

In [ ]:
import seaborn as sns
def plot_regression_results(results):
    """Visualize regression results"""
    plt.figure(figsize=(15, 5))
    
    # Plot predictions vs true values
    plt.subplot(1, 3, 1)
    all_preds = np.concatenate(results['test_preds'])
    all_true = np.concatenate(results['test_true'])
    plt.scatter(all_true, all_preds, alpha=0.3)
    plt.plot([min(all_true), max(all_true)], [min(all_true), max(all_true)], 'r--')
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("Predicted vs True Values")
    
    # Plot metric distributions
    plt.subplot(1, 3, 2)
    metrics = ['mse', 'r2', 'mae']
    metric_names = ['MSE', 'R²', 'MAE']
    for metric, name in zip(metrics, metric_names):
        plt.plot(results[metric], 'o-', label=name)
    plt.xlabel("Fold")
    plt.ylabel("Metric Value")
    plt.title("Metric Values Across Folds")
    plt.legend()
    
    # Plot error distribution
    plt.subplot(1, 3, 3)
    errors = all_preds - all_true
    sns.histplot(errors, kde=True)
    plt.xlabel("Prediction Error")
    plt.title("Error Distribution")
    
    plt.tight_layout()
    plt.show()

In [ ]:
results = evaluate_eot_with_gboost(x, y, n_splits=5, reg_param=0.1, random_state=42)

In [ ]:
plot_regression_results(results)